In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import gc
from tqdm import tqdm
import re
from glob import glob
import numpy as np
import torch
import torchaudio
import fairseq
from fairseq.models.wav2vec import Wav2Vec2Model
from sklearn.model_selection import ShuffleSplit

import sys
sys.path.append('/root/mediazen/speech2blendshape')

from src.utils import plot_result
from src.datasets.new_datamodule import WavDataModule

2022-10-26 14:47:32 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [29]:
device = 'cuda:0'
wav_dir = '/shared/air/shared/youngkim/mediazen/preprocessed/wav'

shape_ggongggong = torch.load('/shared/air/shared/youngkim/mediazen/preprocessed/ggongggong2/shape_ggongggong.pt')
f_names = shape_ggongggong[5]
shape_ggongggong = None

In [31]:
model_path='/shared/air/shared/youngkim/mediazen/pretrained/wav2vec_small_960h.pt'
path, checkpoint = os.path.split(model_path)

# overrides with audio_finetuning task
overrides = {
    "task": 'audio_finetuning',
    "data": path,
}
models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task(
    fairseq.utils.split_paths(model_path),
    arg_overrides=overrides,
    strict=True,
)
base = models[0].w2v_encoder.w2v_model
base.to(device)

Wav2Vec2Model(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
        (3): GELU(approximate=none)
      )
      (1): Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): GELU(approximate=none)
      )
      (2): Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): GELU(approximate=none)
      )
      (3): Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): GELU(approximate=none)
      )
      (4): Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (1): Dr

In [32]:
# # cp = '/shared/air/shared/youngkim/mediazen/pretrained/xlsr_53_56k.pt'
# cp = '/shared/air/shared/youngkim/mediazen/pretrained/wav2vec_small_960h.pt'

# model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp])
# base = model[0]
# base.to(device)

In [33]:
base_outs = []
# xlsr_outs = []
indices = []
feature_lengths = []

base.eval()
# xlsr.eval()
with torch.inference_mode():
    for f_name in tqdm(f_names):
        wav_path = os.path.join(wav_dir, f'{f_name}.wav')
        idx = int(f_name.split('_')[0])
        audio_tensor, sample_rate = torchaudio.load(wav_path)
        audio_tensor = audio_tensor.to(device)
        # base_out = base(audio_tensor)['x']
        base_out = base.forward(audio_tensor, mask=False, features_only=True)['x']
        # xlsr_out = xlsr.forward(audio_tensor, mask=False, features_only=True)['x']
        base_out = base_out.squeeze()
        # xlsr_out = xlsr_out.squeeze().permute(1, 0).contiguous()
        
        base_outs.append(base_out.cpu())
        # xlsr_outs.append(xlsr_out.squeeze().cpu())
        indices.append(idx)
        # assert len(base_out) == len(xlsr_out)
        feature_lengths.append(len(base_out))

        # print(base_out.shape, xlsr_out.shape)
        # break
        audio_tensor, base_out = None, None
        gc.collect()
        torch.cuda.empty_cache()
        

  0%|          | 0/2631 [00:02<?, ?it/s]


In [34]:
base_out.shape

torch.Size([2026, 768])

In [5]:
indices_tensor = torch.IntTensor(indices)
padded_base_tensors = torch.nn.utils.rnn.pad_sequence(base_outs, batch_first=True)
# padded_xlsr_tensors = torch.nn.utils.rnn.pad_sequence(xlsr_outs, batch_first=True)
feature_lengths_tensors = torch.IntTensor(feature_lengths)

In [6]:
base_data = (indices_tensor, padded_base_tensors, feature_lengths_tensors)
# xlsr_data = (indices_tensor, padded_xlsr_tensors, feature_lengths_tensors)

target_dir = '/shared/air/shared/youngkim/mediazen/preprocessed/ggongggong2'

target_base_data = os.path.join(target_dir, 'w2v2_base_ggongggong.pt')
# target_xlsr_data = os.path.join(target_dir, 'w2v2_xlsr_ggongggong.pt')

torch.save(base_data, target_base_data)
# torch.save(xlsr_data, target_xlsr_data)

In [10]:
np.average(feature_lengths_tensors), max(feature_lengths_tensors), min(feature_lengths_tensors)

(2613.671987837324,
 tensor(5840, dtype=torch.int32),
 tensor(1663, dtype=torch.int32))

In [4]:
w2v_ggongggong = torch.load('/shared/air/shared/youngkim/mediazen/preprocessed/ggongggong2/w2v2_base_960h_ggongggong.pt')


In [5]:
indices, padded_bases, feature_lengths = w2v_ggongggong

In [7]:
len(indices), len(padded_bases), len(feature_lengths)

(2631, 2631, 2631)

In [6]:
padded_bases[0].shape

torch.Size([5840, 768])

In [51]:
chk.keys()

dict_keys(['x', 'padding_mask', 'features', 'layer_results'])

In [71]:
base_out['x'].shape == base_out['layer_results'][-1][0].permute(1,0,2).shape

True

In [85]:
model = xlsr.to('cpu')

# create single input
input_wav_0 = torch.randn((1, 2000))
input_wav_1 = torch.randn((1, 3000))

# create batched input
batch_input_wav = torch.zeros((2, 3000))
batch_input_wav[0, :input_wav_0.shape[-1]] = input_wav_0
batch_input_wav[1, :input_wav_1.shape[-1]] = input_wav_1

# create padding mask
padding_mask = torch.zeros((2, 3000), dtype=torch.bool)
padding_mask[0, input_wav_0.shape[-1]:] = True

# run batch & single
output = model.forward(source=input_wav_0, padding_mask=None, mask=False, features_only=True)["x"]
batch_output = model.forward(source=batch_input_wav, padding_mask=padding_mask, mask=False, features_only=True)["x"]

# is equal?
print("Is batched forward and simple forward equal?", torch.allclose(output[0], batch_output[0][:output[0].shape[0],:], atol=1e-1))

Is batched forward and simple forward equal? False


In [86]:
output[0], batch_output[0][:output[0].shape[0],:]

(tensor([[ 0.3966,  0.1221,  0.0295,  ...,  0.1550,  0.1705, -0.3278],
         [ 0.3921,  0.1203,  0.0215,  ...,  0.1574,  0.1746, -0.3288],
         [ 0.3887,  0.1200,  0.0249,  ...,  0.1559,  0.1733, -0.3326],
         [ 0.3903,  0.1239,  0.0242,  ...,  0.1557,  0.1744, -0.3357],
         [ 0.3933,  0.1212,  0.0223,  ...,  0.1552,  0.1740, -0.3317],
         [ 0.3895,  0.1212,  0.0243,  ...,  0.1539,  0.1772, -0.3391]],
        grad_fn=<SelectBackward0>),
 tensor([[ 0.4637,  0.0778, -0.0880,  ..., -0.0238,  0.0912, -0.3438],
         [ 0.4572,  0.0752, -0.0979,  ..., -0.0228,  0.0973, -0.3418],
         [ 0.4502,  0.0738, -0.0926,  ..., -0.0238,  0.0927, -0.3490],
         [ 0.4532,  0.0782, -0.0947,  ..., -0.0255,  0.0959, -0.3499],
         [ 0.4606,  0.0767, -0.0964,  ..., -0.0262,  0.0963, -0.3431],
         [ 0.4532,  0.0760, -0.0912,  ..., -0.0273,  0.0971, -0.3540]],
        grad_fn=<SliceBackward0>))

In [87]:
batch_output[0][output[0].shape[0]:,:]

tensor([[ 0.4599,  0.0775, -0.0920,  ..., -0.0232,  0.0990, -0.3573],
        [ 0.4660,  0.0812, -0.0905,  ..., -0.0213,  0.0968, -0.3606],
        [ 0.4687,  0.0842, -0.0924,  ..., -0.0227,  0.0966, -0.3614]],
       grad_fn=<SliceBackward0>)